In [1]:
import numpy as np
import cv2
import torch 
import torchvision 
from engine import train_one_epoch, evaluate
from data_reading import WindowDataset
import utils
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter


In [2]:
def get_model():
    # Initialize Model, from pytorch tutorial: https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)
    num_classes = 2 # window + background
    in_features = model.roi_heads.box_predictor.cls_score.in_features # Number of input features for the classifier
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    return model.double()

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'running on device {device}')
model = get_model()
model.to(device)

running on device cuda


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [4]:
params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(params, lr = 0.00001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.1)

In [ ]:
dataset = WindowDataset()
dataset[0]

In [5]:
train = DataLoader(WindowDataset(), batch_size=1, shuffle=True, collate_fn=utils.collate_fn)
print(next(iter(train)))

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
((tensor([[[0.9765, 0.9608, 0.9294,  ..., 0.9608, 0.9569, 0.9490],
         [0.9804, 0.9412, 0.9216,  ..., 0.9529, 0.9412, 0.9333],
         [0.9647, 0.9176, 0.9412,  ..., 0.9373, 0.9255, 0.9137],
         ...,
         [0.7176, 0.7333, 0.7490,  ..., 0.6549, 0.6588, 0.6392],
         [0.6980, 0.7176, 0.7373,  ..., 0.6471, 0.6510, 0.6471],
         [0.6863, 0.7059, 0.7294,  ..., 0.6471, 0.6549, 0.6235]],

        [[0.9961, 0.9804, 0.9490,  ..., 0.9333, 0.9294, 0.9216],
         [1.0000, 0.9608, 0.9412,  ..., 0.9255, 0.9137, 0.9059],
         [0.9843, 0.9373, 0.9608,  ..., 0.9098, 0.8980, 0.8863],
         ...,
         [0.7059, 0.7216, 0.7373,  ..., 0.7529, 0.7569, 0.7294],
         [0.6863, 0.7059, 0.7255,  ..., 0.7569, 0.7490, 0.7373],
         [0.6745, 0.6941, 0.7176,  ..., 0.7569, 0.7529, 0.7216]],

        [[0.9804, 0.9647, 0.9333,  ..., 0.8941, 0.8902, 0.8824],
         [0.9843, 0.9451, 0.9255,  ...

In [ ]:
print(next(iter(train))[1][0]['labels'].dtype)

In [ ]:
print(device)
evaluate(model, train, device, print_freq=10)

In [6]:
num_epochs = 40
writer = SummaryWriter()
for epoch in range(num_epochs):
    # Train for one epoch, while printing every 10 iterations
    train_one_epoch(writer, model, optimizer, train, device, epoch, print_freq=100)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, train, device, print_freq=100)

print("DONE")

Epoch: [0]  [   0/1029]  eta: 3:14:19  lr: 0.000010  loss: 1.2922 (1.2922)  loss_classifier: 0.8957 (0.8957)  loss_box_reg: 0.0562 (0.0562)  loss_objectness: 0.3018 (0.3018)  loss_rpn_box_reg: 0.0384 (0.0384)  time: 11.3313  data: 0.0010  max mem: 2046
